In [2]:
import pandas as pd
import re
import datetime


In [3]:
df = pd.read_csv("data.csv")
#df

# column name: location

In [ ]:
df["location"]

# Column Name: Make

In [4]:
df["make"]


0         \r\n,Audi ,\r\n
1         \r\n,Audi ,\r\n
2         \r\n,Audi ,\r\n
3         \r\n,Audi ,\r\n
4         \r\n,Audi ,\r\n
               ...       
62761    \r\n,Volvo ,\r\n
62762    \r\n,Volvo ,\r\n
62763    \r\n,Volvo ,\r\n
62764    \r\n,Volvo ,\r\n
62765    \r\n,Volvo ,\r\n
Name: make, Length: 62766, dtype: object

In [271]:
df["make"]=df["make"].str.strip(',\n\r')
df["make"]

0         Audi 
1         Audi 
2         Audi 
3         Audi 
4         Audi 
          ...  
62761    Volvo 
62762    Volvo 
62763    Volvo 
62764    Volvo 
62765    Volvo 
Name: make, Length: 62766, dtype: object

# Column Name: model

In [6]:
df["model"]


0         \r\n,A3,\r\n
1         \r\n,A3,\r\n
2         \r\n,A1,\r\n
3         \r\n,A1,\r\n
4         \r\n,A3,\r\n
             ...      
62761    \r\n,S60,\r\n
62762    \r\n,S60,\r\n
62763    \r\n,S60,\r\n
62764    \r\n,S40,\r\n
62765    \r\n,S60,\r\n
Name: model, Length: 62766, dtype: object

In [272]:
df["model"]=df["model"].str.strip(',\n\r')
df["model"]

0         A3
1         A3
2         A1
3         A1
4         A3
        ... 
62761    S60
62762    S60
62763    S60
62764    S40
62765    S60
Name: model, Length: 62766, dtype: object

# column name: short_description

In [8]:
df['short_description']= df['short_description'].str.strip("\n,")
df['short_description']

0                                2.0 FSI Ambition Pro Line
1        Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...
2          1.4 TFSI Sportback Pro-Line Aut. S-Tronic Navi.
3        S-Line | Automaat | Navigatie | Cruise-/Climat...
4        Sportback 1.5 TFSI Sport | Dealer Onderhouden ...
                               ...                        
62761                            2.4 Edition APK/LMV/Airco
62762          2.4 Kinetic LPG-G3 NAP/LEER/NAVI/DAK/NW APK
62763    2.0T Edition,1e Eigenaar,LPG G3,Onderbouw,Navi...
62764                                                  2.4
62765        2.4 Airco, Cruise Control, Stuurbekrachtiging
Name: short_description, Length: 62766, dtype: object

# Column name: price

In [9]:
df['price'] = df['price'].apply(lambda x: '€ ' + re.sub(r'[^\d.,]+', '', x).replace(',', '.').replace('.', '', 1).rstrip('.'))
df['price']

0         € 4.450
1        € 12.950
2        € 12.450
3        € 14.945
4        € 24.900
           ...   
62761     € 1.799
62762     € 1.999
62763     € 3.449
62764     € 1.650
62765       € 944
Name: price, Length: 62766, dtype: object

# Column Name : mileage

In [10]:
mileage_1 = []
for i in df['mileage']:
    result = re.search(r'\d{1,3}(?:,\d{3})*', i)
    if result:
        mileage_1.append(result.group(0))
    else:
        mileage_1.append(None)

mileage_son = pd.DataFrame(mileage_1,columns=["mileage"])

df["mileage"] = mileage_son["mileage"]

In [11]:
df.rename(columns = {'mileage':'mileage(km)'}, inplace = True)  # in place of every line, we write 'km' to column name as unit 
df["mileage(km)"].head(10)  

0    191,066
1     94,758
2    145,102
3    119,666
4     55,815
5     47,536
6     94,530
7    139,703
8    103,626
9    100,558
Name: mileage(km), dtype: object

# colum name : gearbox  #Gearbox sütunu ile karşılaştırıldı 

In [12]:
df['gearbox'] = df['gearbox'].str.replace(r'[\d,]+', '', regex=True).str.strip()
df['gearbox']

0        Automatic
1        Automatic
2        Automatic
3        Automatic
4        Automatic
           ...    
62761       Manual
62762       Manual
62763       Manual
62764       Manual
62765       Manual
Name: gearbox, Length: 62766, dtype: object

# Gearbox sütunu ile karşılaştırıldı 

In [274]:

df['Gearbox'] = df['Gearbox'].str.strip("\n,")
df['gearbox'] = df['gearbox'].fillna("None")
df.loc[(df['gearbox'] != 'None'), ['gearbox', 'Gearbox']]

,gearbox,Gearbox
0,Automatic,Automatic
1,Automatic,Automatic
2,Automatic,Automatic
3,Automatic,Automatic
4,Automatic,Automatic
...,...,...
62761,Manual,Manual
62762,Manual,Manual
62763,Manual,Manual
62764,Manual,Manual


# colum name : registration

In [275]:
df['registration']

0        08/2003
1        10/2013
2        06/2015
3        10/2015
4        10/2017
          ...   
62761    12/2000
62762    02/2005
62763    08/2004
62764    01/2004
62765    09/2001
Name: registration, Length: 62766, dtype: object

# Column name : fuel_type  #Fuel_type sütunu ile karşılaşırıldı

In [276]:
df['fuel_type'] = df['fuel_type'].str.strip("\n\r,")
df['Fuel_type'] = df['Fuel_type'].str.strip("\n\r,")
df['fuel_type'] = df['fuel_type'].fillna("None")
df.loc[(df['fuel_type'] != 'None'), ['fuel_type', 'Fuel_type']]

,fuel_type,Fuel_type
0,Gasoline,Super 95
1,Gasoline,Gasoline
2,Gasoline,Gasoline
3,Gasoline,Gasoline
4,Gasoline,Super 95
...,...,...
62761,LPG,LPG
62762,LPG,LPG
62763,LPG,LPG
62764,LPG,Liquid petroleum gas (LPG)


# Column name : power     #Power sütunu ile karşılaştırıldı 

In [277]:
df['power'] = df['power'].str.strip("\n\r,")
df['Power'] = df['Power'].str.strip("\n\r,")
df['power'] = df['power'].fillna("None")
df.loc[(df['power'] != 'None'), ['power', 'Power']]

,power,Power
0,110 kW (150 hp),110 kW (150 hp)
1,92 kW (125 hp),92 kW (125 hp)
2,92 kW (125 hp),92 kW (125 hp)
3,92 kW (125 hp),92 kW (125 hp)
4,110 kW (150 hp),110 kW (150 hp)
...,...,...
62761,125 kW (170 hp),125 kW (170 hp)
62762,103 kW (140 hp),103 kW (140 hp)
62763,132 kW (179 hp),132 kW (179 hp)
62764,103 kW (140 hp),103 kW (140 hp)


# Column Name : seller

In [278]:
df['seller'] = df['seller'].str.lstrip("[', [' \s\S\t n ")
df['seller'] = df['seller'].str.rstrip("[', [] ' \s\S\t n")  


#seller has three not related unique values(electric, diesel , Gasoline)
df["seller"].replace("Diesel", None, inplace = True) 
df["seller"].replace("Electric", None, inplace = True) 
df["seller"].replace("Gasoline", None, inplace = True) 
df["seller"].unique()

array(['Dealer', 'Private seller', None], dtype=object)

# Column name: body_type

In [16]:
df['body_type'] = df['body_type'].str.replace(',', '').str.strip()
df['body_type']

0        Compact
1        Compact
2        Compact
3        Compact
4        Compact
          ...   
62761      Sedan
62762      Sedan
62763      Sedan
62764      Sedan
62765      Sedan
Name: body_type, Length: 62766, dtype: object

# Column name: type

In [18]:
df['type'] = df['type'].str.replace(',', '').str.strip()
df['type']

0        Used
1        Used
2        Used
3        Used
4        Used
         ... 
62761    Used
62762    Used
62763    Used
62764    Used
62765    Used
Name: type, Length: 62766, dtype: object

# Column name: drivetrain

In [53]:
df['drivetrain'] = df['drivetrain'].str.replace(',', '').str.strip()
df['drivetrain']

0        Front
1        Front
2         None
3        Front
4        Front
         ...  
62761    Front
62762    Front
62763    Front
62764    Front
62765    Front
Name: drivetrain, Length: 62766, dtype: object

# Column Name: seats

In [202]:
df["seats"]

0        \n5
1        \n5
2        \n5
3        \n4
4        \n5
        ... 
62761    \n5
62762    \n5
62763    \n5
62764    \n5
62765    \n5
Name: seats, Length: 62766, dtype: object

In [203]:
df["seats"]=df["seats"].str.strip(',\n')

In [204]:
df["seats"].replace("None", "0", inplace = True) #None values may stay as None. There are no car without seat

In [205]:
df["seats"]=df["seats"].astype(int)

df["seats"].head()

0    5
1    5
2    5
3    4
4    5
Name: seats, dtype: int32

# Column Name: doors

In [200]:
df["doors"].unique()

array(['\n3', '\n5', '\n4', 'None\n', '\n2', '\n6', '\n1'], dtype=object)

In [201]:

df["doors"]=df["doors"].str.strip(',\n')
df["doors"].replace("None", "0", inplace = True) #None values may stay as None. There are no car without door
df["doors"]=df["doors"].astype(int)
df["doors"].head()

0    3
1    5
2    5
3    3
4    5
Name: doors, dtype: int32

# Column Name: country_version

In [30]:
df['country_version'] = df['country_version'].str.split(',').str[-1].str.strip()
df['country_version']

0               None
1               None
2        Netherlands
3               None
4               None
            ...     
62761           None
62762           None
62763           None
62764           None
62765           None
Name: country_version, Length: 62766, dtype: object

# Column name : offer_number #formatını mentöre soralım

In [356]:
df["offer_number"]= df["offer_number"].str.strip(',\n')
df["offer_number"]

0        SJ-112-R
1          809725
2         2601307
3        L-847-SK
4        PZ-744-H
           ...   
62761    52-XX-LB
62762    06-RB-XS
62763    75-PK-RN
62764        None
62765    26-HN-LS
Name: offer_number, Length: 62766, dtype: object

# Column name : warranty

In [32]:
df['warranty'] = df['warranty'].str.replace(',', '').str.strip()
df['warranty']

0        6 months
1            None
2        6 months
3        6 months
4        6 months
           ...   
62761        None
62762        None
62763        None
62764        None
62765        None
Name: warranty, Length: 62766, dtype: object

# Column name : Mileage    #Data içinde 2 adet Mileage sütunu mevcut ve bu sütunda bilgi olmadığından drop yapılacak

In [199]:
df['Mileage'].unique()

array(["[['\\n'], ['']]", "[['\\n'], ['None']]"], dtype=object)

# Column name: first_registration

In [196]:
df['first_registration'].head()


0    08/2003
1    10/2013
2    06/2015
3    10/2015
4    10/2017
Name: first_registration, dtype: object

# Column Name : production_date  ve registration değerleri birbiri ile uyumlu

In [ ]:
# df['production_date'] = pd.to_datetime(df['production_date'], format='%Y')

#Producton_date ve registration arasinda bag var. Yillar ayni. Burdan doldurabiliriz
df['production_date'] = df['production_date'].fillna("None")
df.loc[(df['production_date'] != 'None'), ['production_date', 'registration']]


,production_date,registration
2,2015,06/2015
7,2011,07/2011
8,2012,08/2012
14,2012,05/2012
26,2013,07/2013
...,...,...
62696,2007,03/2007
62708,2010,09/2010
62714,2007,07/2007
62719,2005,02/2005


# Column Name : general_inspection #bu veride ne yapılacak

In [358]:
df["general_inspection"].value_counts()

general_inspection
New        15532
06/2022     2169
03/2022     2025
07/2022     1936
05/2022     1930
           ...  
11/2014        1
09/2017        1
08/2018        1
06/2019        1
12/2019        1
Name: count, Length: 93, dtype: int64

# Column Name : last_service               #column içerisinde sadece nan değeri var

In [191]:
df['last_service'].unique()

array([nan])

# Column Name: full_service_history

In [37]:
df['full_service_history'] = df['full_service_history'].str.replace(',', '').str.strip()
df['full_service_history']

0         Yes
1         Yes
2         Yes
3         Yes
4         Yes
         ... 
62761     Yes
62762     Yes
62763     Yes
62764    None
62765    None
Name: full_service_history, Length: 62766, dtype: object

# Column Name : non_smoker_vehicle

In [40]:
df['non_smoker_vehicle'] = df['non_smoker_vehicle'].str.replace(',', '').str.strip()
df['non_smoker_vehicle']

0        None
1        None
2         Yes
3        None
4        None
         ... 
62761    None
62762    None
62763    None
62764    None
62765    None
Name: non_smoker_vehicle, Length: 62766, dtype: object

# Column Name : previous_owner

In [186]:
df['previous_owner'] = df['previous_owner'].str.lstrip("[' \s\S\t n ',]")
df['previous_owner'] = df['previous_owner'].str.rstrip("['\s,]")
df['previous_owner'].unique()

array(['None', '3', '2', '1', '5', '7', '9', '4', '6', '8', '12', '10'],
      dtype=object)

# Column Name : last_timing_belt_change

In [ ]:
df['last_timing_belt_change'] = df['last_timing_belt_change'].str.lstrip(" [',] '\a \s\S\t\r n ' ")
df['last_timing_belt_change'] = df['last_timing_belt_change'].str.rstrip(" [',] '\a \s\S\t\r n ' ")

df['last_timing_belt_change'].unique()

array(['None', '07/2021', '03/2021', '05/2019', '06/2021', '10/2019',
       '02/2018', '06/2020', '06/2017', '03/2019', '08/2019', '04/2019',
       '04/2021', '10/2020', '11/2021', '07/2018', '12/2018', '11/2019',
       '04/2016', '05/2021', '08/2018', '10/2021', '01/2021', '07/2017',
       '07/2019', '04/2020', '12/2021', '09/2017', '09/2021', '01/2020',
       '09/2020', '03/2020', '08/2021', '05/2016', '01/2015', '01/2017',
       '04/2018', '01/2016', '09/2015', '12/2019', '07/2020', '12/2020',
       '08/2020', '05/2020', '07/2011', '02/2020', '10/2017', '06/2019',
       '09/2019', '10/2016', '11/2018', '10/2018', '08/2017', '01/2019',
       '11/2015', '02/2021', '02/2019', '11/2020', '05/2017', '05/2018',
       '01/2018', '03/2018', '06/2013', '11/2000', '01/1900', '09/2018',
       '02/2017', '09/2016', '12/2017', '06/2018', '05/2015', '04/2014',
       '04/2017', '11/2017', '06/2016', '12/2015', '10/2015', '02/2015',
       '02/2016', '06/2014', '04/2015', '08/2016'], dt

# Column Name : Power

In [361]:
df["power"].value_counts()

power
[None, '']         5333
110 kW (150 hp)    3677
88 kW (120 hp)     2518
100 kW (136 hp)    2190
90 kW (122 hp)     2174
                   ... 
378 kW (514 hp)       1
282 kW (383 hp)       1
470 kW (639 hp)       1
451 kW (613 hp)       1
10 kW (14 hp)         1
Name: count, Length: 361, dtype: int64

# Column Name: gearbox   aynı sütundan var

# Column Name : engine_size

In [ ]:
# kolonun ici düzenlendi, none ifade = 0 a ayarlandi.
# ancak cc biriminden dolayi int cevrilmiyor, bu yüzden de mean() vs islemlr de yapilamiyor
#??????

In [279]:
df["engine_size"]= df["engine_size"].str.strip(',\n,')
df["engine_size"]= df["engine_size"].replace("None", "0")
df["engine_size"]= df["engine_size"].str.replace("cc", "")

df["engine_size"].head()

0    1,984 
1    1,395 
2    1,395 
3    1,395 
4    1,495 
Name: engine_size, dtype: object

# Column Name : gears

In [280]:
df["gears"]=df["gears"].str.strip(',\n,')
df["gears"].unique()

array(['None', '7', '6', '8', '5', '1', '9', '4', '2', '3'], dtype=object)

bi iki koda gerek var mi pek emin degilim,
Data missing de konusabiliriz!!!
df["gears"].replace("None", "0", inplace = True)
df["gears"]=df["gears"].astype(int)

# Column Name: cylinders

In [183]:
df["cylinders"]=df["cylinders"].str.strip(',\n,')
df["cylinders"].unique()
#df["cylinders"]

array(['4', '3', '6', 'None', '5', '8', '10', '12', '1', '2', '7'],
      dtype=object)

# Column Name : empty_weight

In [180]:
df["empty_weight"]=df["empty_weight"].str.strip('[\n,')
df["empty_weight"]=df["empty_weight"].str.replace(' kg', '')
df["empty_weight"].head()

0    1,280
1    1,200
2     None
3    1,105
4    1,215
Name: empty_weight, dtype: object

# Column Name: Fuel_type

In [179]:
df["Fuel_type"]=df["Fuel_type"].str.strip('[\n,')
df["Fuel_type"].head(15)

0     Super 95
1     Gasoline
2     Gasoline
3     Gasoline
4     Super 95
5     Super 95
6     Gasoline
7     Gasoline
8     Gasoline
9     Super 95
10    Gasoline
11    Super 95
12    Gasoline
13    Super 95
14    Gasoline
Name: Fuel_type, dtype: object

# Column name : fuel_consumption

In [177]:
df['fuel_consumption'] = df['fuel_consumption'].str.lstrip(" [', \s\S\t\r\n,n ' ")
df['fuel_consumption'] = df['fuel_consumption'].str.rstrip("[', '] \s\S\t\r\n,\\n  ")
df['fuel_consumption'] = df['fuel_consumption'].str.replace("'","")
df['fuel_consumption']

0                                                     None
1                                     4.9 l/100 km (comb.)
2        0 l/100 km (comb.), 0 l/100 km (city), 0 l/100...
3        4.8 l/100 km (comb.), 5.9 l/100 km (city), 4.2...
4        5 l/100 km (comb.), 6.2 l/100 km (city), 4.3 l...
                               ...                        
62761                                                 None
62762                                                 None
62763    9.1 l/100 km (comb.), 12.6 l/100 km (city), 7 ...
62764                                                 None
62765    8.4 l/100 km (comb.), 11.6 l/100 km (city), 6....
Name: fuel_consumption, Length: 62766, dtype: object

# Column Name: co2_emissions

In [176]:
df['co2_emissions'] = df['co2_emissions'].str.strip(",\n")
df["co2_emissions"] = df["co2_emissions"].str.rstrip(" [',] \s\S\t\r\n n' ")
df["co2_emissions"] = df["co2_emissions"].str.lstrip(" [', \s\S\t\r\n ' ")
df['co2_emissions']


0                        
1                    None
2                    None
3        111 g/km (comb.)
4        113 g/km (comb.)
               ...       
62761                    
62762                    
62763    215 g/km (comb.)
62764                None
62765    207 g/km (comb.)
Name: co2_emissions, Length: 62766, dtype: object

# Column Name : emission_class

In [166]:
df['emission_class'] = df['emission_class'].str.strip(",\n")
df['emission_class']

0        Euro 4
1          None
2          None
3        Euro 6
4        Euro 6
          ...  
62761      None
62762      None
62763    Euro 3
62764      None
62765    Euro 3
Name: emission_class, Length: 62766, dtype: object

# Column Name: emissions_sticker

In [155]:
df["emissions_sticker"].unique()

array(['None', '4 (Green)', '3 (Yellow)', '1 (No sticker)', '', '2 (Red)'],
      dtype=object)

In [153]:
df["emissions_sticker"] = df["emissions_sticker"].str.rstrip(" [',] \s\S\t\r\n ' ")
df["emissions_sticker"] = df["emissions_sticker"].str.lstrip(" [',] \s\S\t\r\n ' ")
df["emissions_sticker"] = df["emissions_sticker"].str.strip(" ")
df["emissions_sticker"] = df["emissions_sticker"].str.replace("\\n","")
df["emissions_sticker"].unique()

array(['None', '4 (Green)', '3 (Yellow)', '1 (No sticker)', '', '2 (Red)'],
      dtype=object)

# Column Name : Comfort & Convenience

In [138]:
df.rename({"\nComfort & Convenience\n":"comfort_convenience"}, inplace = True, axis = 1) # sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["comfort_convenience"] = df["comfort_convenience"].str.rstrip(" [', \s\S\t\r n ' ")
df["comfort_convenience"] = df["comfort_convenience"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["comfort_convenience"] = df["comfort_convenience"].str.replace("'","")
df["comfort_convenience"]

0        Air conditioning, Armrest, Automatic climate c...
1        Air conditioning, Automatic climate control, E...
2        Air conditioning, Armrest, Automatic climate c...
3        Air conditioning, Armrest, Automatic climate c...
4        Air conditioning, Armrest, Electrical side mir...
                               ...                        
62761    Air conditioning, Armrest, Automatic climate c...
62762    Air conditioning, Automatic climate control, C...
62763    Air conditioning, Armrest, Automatic climate c...
62764    Air conditioning, Armrest, Automatic climate c...
62765    Air conditioning, Armrest, Automatic climate c...
Name: comfort_convenience, Length: 62766, dtype: object

# Column Name : Entertaiment & Media

In [132]:
df['\nEntertainment & Media\n'].unique()

array(["[['CD player', 'Radio']]",
       "[['Bluetooth', 'CD player', 'On-board computer', 'USB']]",
       "[['CD player', 'MP3', 'On-board computer', 'Radio']]", ...,
       "[['Apple CarPlay', 'CD player', 'Digital cockpit', 'On-board computer', 'Radio']]",
       "[['Apple CarPlay', 'CD player', 'On-board computer', 'Radio', 'Sound system']]",
       "[['Android Auto', 'Apple CarPlay', 'Bluetooth', 'CD player', 'Digital cockpit', 'Digital radio', 'Hands-free equipment', 'Induction charging for smartphones', 'Integrated music streaming', 'On-board computer', 'Radio', 'Sound system', 'USB']]"],
      dtype=object)

In [137]:
df.rename({"\nEntertainment & Media\n":"entertainment_media"},inplace = True, axis = 1)# sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["entertainment_media"] = df["entertainment_media"].str.rstrip(" [',] \s\S\t\r n ' ")
df["entertainment_media"] = df["entertainment_media"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["entertainment_media"] =  df["entertainment_media"].str.replace("'","")
df["entertainment_media"].unique()

array(['CD player, Radio', 'Bluetooth, CD player, On-board computer, USB',
       'CD player, MP3, On-board computer, Radio', ...,
       'Apple CarPlay, CD player, Digital cockpit, On-board computer, Radio',
       'Apple CarPlay, CD player, On-board computer, Radio, Sound system',
       'Android Auto, Apple CarPlay, Bluetooth, CD player, Digital cockpit, Digital radio, Hands-free equipment, Induction charging for smartphones, Integrated music streaming, On-board computer, Radio, Sound system, USB'],
      dtype=object)

# Column Name : Safety & Security

In [131]:
df.rename({"\nSafety & Security\n":"safety_security"},inplace = True, axis = 1)# sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["safety_security"] = df["safety_security"].str.rstrip(" [', \s\S\t\r n ' ")
df["safety_security"] = df["safety_security"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["safety_security"] =  df["safety_security"].str.replace("'","")
df["safety_security"]

0        ABS, Alarm system, Central door lock, Central ...
1        Alarm system, Central door lock, Central door ...
2        ABS, Central door lock with remote control, Dr...
3        ABS, Alarm system, Bi-Xenon headlights, Centra...
4        ABS, Alarm system, Bi-Xenon headlights, Centra...
                               ...                        
62761    ABS, Central door lock, Central door lock with...
62762    ABS, Central door lock, Central door lock with...
62763    ABS, Central door lock, Central door lock with...
62764    ABS, Central door lock with remote control, Dr...
62765    ABS, Alarm system, Central door lock, Central ...
Name: safety_security, Length: 62766, dtype: object

# Column Name : extras

In [130]:
df["extras"] = df["extras"].str.rstrip(" [',] \s\S\t\r n ' ")
df["extras"] = df["extras"].str.lstrip(" [',] '\a \s\S\t\r n ' ")
df["extras"] =  df["extras"].str.replace("'","")
df["extras"]

0                                              Alloy wheel
1                                       Alloy wheels (16")
2                                       Alloy wheels (15")
3        Alloy wheels (17"), Shift paddles, Sport seats...
4         Alloy wheels (17"), Sport seats, Sport suspensio
                               ...                        
62761                                   Alloy wheels (15")
62762                                   Alloy wheels (15")
62763    Alloy wheels (17"), Automatically dimming inte...
62764    Alloy wheels (16"), Automatically dimming inte...
62765                                          Alloy wheel
Name: extras, Length: 62766, dtype: object

# Column Name: manifacturer_colour    içeriği tamamen nan değeriyle dolu

In [129]:
df['\nmanufacturer_color'].unique()

array([nan])

# Column Name: colour

In [ ]:
df.rename(columns = {'\ncolour':'colour'}, inplace = True)
df["colour"].unique()

array(['\n4, Grey', '\n4, White', '\n4, Black', '\n4, Red', '\n4, Silver',
       '\n4, Brown', '\n4, Blue', '\n4, Violet', 'None', '\n4, Beige',
       '\n4, Yellow', '\n4, Green', '\n4, Orange', '\n4, Bronze',
       '\n4, Gold'], dtype=object)

In [ ]:
df["colour"]=df["colour"].str.strip(',\n4,')
df["colour"].head()
#1048 none var

0      Grey
1     White
2      Grey
3     Black
4      Grey
Name: colour, dtype: object

# Column Name : paint   (arabanın boyanıp boyanmadığını ifade ediyor. Renk belirtilenler boyanmış belirtilmeyenler boyanmamış. Bundan dolayı rengin yanındaki paint ifadesini gözardı edebiliriz.)

In [127]:
df.rename(columns = {'\npaint':'paint'}, inplace = True)
df["paint"].unique()
df['paint']


0        \npaint, Metallic
1                      NaN
2        \npaint, Metallic
3                      NaN
4        \npaint, Metallic
               ...        
62761                  NaN
62762                  NaN
62763    \npaint, Metallic
62764    \npaint, Metallic
62765    \npaint, Metallic
Name: paint, Length: 62766, dtype: object

In [128]:
df["paint"]=df["paint"].str.replace('\npaint,','')
df['paint']

0         Metallic
1              NaN
2         Metallic
3              NaN
4         Metallic
           ...    
62761          NaN
62762          NaN
62763     Metallic
62764     Metallic
62765     Metallic
Name: paint, Length: 62766, dtype: object

# Column Name : upholstery_colour

In [122]:
df[' upholstery_colour '].unique()

array(['\nupholstery, Black', nan, '\nupholstery, Grey',
       '\nupholstery, Brown', '\nupholstery, Other',
       '\nupholstery, Beige', '\nupholstery, Red', '\nupholstery, Yellow',
       '\nupholstery, Blue', '\nupholstery, White', '\nupholstery, Green',
       '\nupholstery, Orange'], dtype=object)

In [123]:
df.rename(columns={' upholstery_colour ':'upholstery_colour'},inplace = True)
df["upholstery_colour"]=df["upholstery_colour"].str.replace('\nupholstery,','')
df["upholstery_colour"].unique()

array([' Black', nan, ' Grey', ' Brown', ' Other', ' Beige', ' Red',
       ' Yellow', ' Blue', ' White', ' Green', ' Orange'], dtype=object)

# Column Name : upholstery

In [120]:
df[" upholstery "].unique()

array(['\n8, Cloth', nan, '\n8, Part leather', '\n8, Full leather',
       '\n8, alcantara', '\n8, Other', '\n8, Velour'], dtype=object)

In [121]:
df.rename(columns={' upholstery ':'upholstery'},inplace = True)
df["upholstery"]=df["upholstery"].str.replace('\n8,','')
df["upholstery"].unique()

array([' Cloth', nan, ' Part leather', ' Full leather', ' alcantara',
       ' Other', ' Velour'], dtype=object)